# Crypto Challenge Set 1

This is the **qualifying set**. We picked the exercises in it to ramp developers up gradually into coding cryptography, but also to verify that we were working with people who were ready to write code.

This set is **relatively easy**. With one exception, most of these exercises should take only a couple minutes. But don't beat yourself up if it takes longer than that. It took Alex two weeks to get through the set!

If you've written any crypto code in the past, you're going to feel like skipping a lot of this. **Don't skip them**. At least two of them (we won't say which) are important stepping stones to later attacks.

1. Convert hex to base64
1. Fixed XOR
1. Single-byte XOR cipher
1. Detect single-character XOR
1. Implement repeating-key XOR
1. Break repeating-key XOR
1. AES in ECB mode
1. Detect AES in ECB mode

## 1 - Convert hex to base64

String: "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
Should produce: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


In [ ]:
import base64
ptext_hex = "0x49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
ptext_bytes = bytes.fromhex(ptext_hex[2:])
print( base64.b64encode(ptext_bytes) )

## 2 - Fixed XOR
Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:
`1c0111001f010100061a024b53535009181c`

... after hex decoding, and when XOR'd against:
`686974207468652062756c6c277320657965`

... should produce:
`746865206b696420646f6e277420706c6179`

In [ ]:
hex_a = 0x1c0111001f010100061a024b53535009181c
hex_b = 0x686974207468652062756c6c277320657965
def xor(hex_a, hex_b):
    hex_c = hex_a ^ hex_b
    return hex_c
print( hex(xor(hex_a, hex_b)) )

## 3 - Single-byte XOR cipher
The hex encoded string:
`1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736`
... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.    

In [ ]:
cbytes = bytes.fromhex("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")
print(cbytes.decode("ASCII"))

k = {}

for char in string.ascii_lowercase+string.ascii_uppercase:
    tptext = ""
    for CB in cbytes:
        tptext += chr(CB^ord(char) )
    k.update({char:tptext})
    print("{:<3}{}".format(char,tptext.replace("\n","+").replace("\r","+")))

## 4 - Detect single-character XOR
One of the 60-character strings in this file has been encrypted by single-character XOR.

Find it.

(Your code from #3 should help.)

In [ ]:
CCC = string.ascii_lowercase+string.ascii_uppercase+"0123456789 .-,"
with open("4.txt","r") as fh:
    for line in fh.read().strip().split():
        cbytes = bytes.fromhex(line)
        K = {}
        for c in range(0,256):
            tptext = "".join(chr(CB^c) for CB in cbytes)
            K.update({c:tptext})
    
        for k in K.keys():
            K[k] = K[k].replace("\n","").replace("\r","")
            i = 0
            for c in K[k]:
                if c not in CCC:
                    i += 1
            if i < 1:
                print(line, cbytes.hex())
                print("{} {} {}".format(" "*30, k, K[k]))

## 5 - Implement repeating-key XOR
Here is the opening stanza of an important work of the English language:

Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal
Encrypt it, under the key "ICE", using repeating-key XOR.

In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.

It should come out to:
- `0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272`
- `a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f`


Encrypt a bunch of stuff using your repeating-key XOR function. Encrypt your mail. Encrypt your password file. Your .sig file. Get a feel for it. I promise, we aren't wasting your time with this.

In [ ]:
KEY = "ICE".encode("ASCII")

PTEXT = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
PTEXT = PTEXT.encode("ASCII")

print(PTEXT)
print(KEY)

def repeat_xor(key, data):
    i = 0
    out = b""
    for PB in data:
        out += (PB^key[i%len(key)]).to_bytes(1, "big")
        i+=1
    return out

a = repeat_xor(KEY, PTEXT)
a.hex()

6 - Break repeating-key XOR
===


In [ ]:
def str2bin(C):
    if type(C) is str:
        return "{}{}".format("0"*(8-len(C)), C)
    elif type(C) is bytes:
        res = [str2bin(str(bin(c))[2:]) for c in C]
        return "".join(res)

def aXORb(A, B):
    C = str(bin(A^B))[2:]
    return C

def hamming_distance(A, B):
    if len(A) != len(B):
        return -1
    d = 0
    for i in range(0, len(A)):
        for c in aXORb(A[i],B[i]):
            if c=="1":
                d+=1
    return d

print(hamming_distance( b"this is a test", b"wokka wokka!!!" ))
print(str2bin(b"this is a test"))


import base64
with open("6.txt","r") as fh:
    tmp = fh.read().replace("\n","")
cipher = base64.b64decode(tmp)

KEYSIZE_MAX = 40
KEYSIZE_MIN = 2

K = []
K_PROB = [-1, 30*10]

for KEYSIZE in range(KEYSIZE_MIN, KEYSIZE_MAX):
    hd = hamming_distance( cipher[:KEYSIZE], cipher[KEYSIZE:KEYSIZE*2] )
    K.append( [KEYSIZE, hd, hd/KEYSIZE] )

min_index = 0
for index,k in enumerate(K):
    if K[min_index][2] >= k[2]:
        min_index = index
print(f"Min keysize {K[min_index][0]} -  {K[min_index][1]} -  {K[min_index][2]}")
ks = K[min_index][0]
m = 0
cipher_ks = []
ki=0
print(len(cipher))
while ki+ks < len(cipher):
    cipher_ks.append( cipher[ki : ki+ks] )
    m += 1
    ki += ks

cipher_blk = []
for i in range(ks):
    cipher_blk.append( [blk[i] for blk in cipher_ks] )

import string
alphabet = string.ascii_uppercase+string.ascii_lowercase+string.digits
for blk in cipher_blk:
    blk_str = "".join([chr(b) for b in blk])
    print(blk_str)
    for a in alphabet:
        print(
            hamming_distance(
                            "".join(),
                            "{}".format(a*len(blk))
                        )
        )